# Measuring Underserved Areas in the MTA

In [23]:
from src.load_mta_dataset import DATASETS, download_mta_data, load_mta_data

# Download and load MTA data
download_mta_data()
load_mta_data()

File data/gtfs_subway.zip already exists. Skipping download.
File data/gtfsmnr.zip already exists. Skipping download.
File data/gtfs_m.zip already exists. Skipping download.


/Users/Deena/Desktop/DS 501/ds-501-cs4/.venv/lib/python3.13/site-packages/gtfs_kit/feed.py:394: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



In [24]:
print(DATASETS)

[GTFSDataset(id='subway', url='https://rrgtfsfeeds.s3.amazonaws.com/gtfs_subway.zip', path=PosixPath('data/gtfs_subway.zip'), gk_feed=<gtfs_kit.feed.Feed object at 0x176755f30>), GTFSDataset(id='metro_north_railroad', url='https://rrgtfsfeeds.s3.amazonaws.com/gtfsmnr.zip', path=PosixPath('data/gtfsmnr.zip'), gk_feed=<gtfs_kit.feed.Feed object at 0x176756060>), GTFSDataset(id='manhattan_bus', url='https://rrgtfsfeeds.s3.amazonaws.com/gtfs_m.zip', path=PosixPath('data/gtfs_m.zip'), gk_feed=<gtfs_kit.feed.Feed object at 0x123028dd0>)]


In [25]:
subway_data = DATASETS[0]
subway_data.gk_feed.describe()

,indicator,value
0,agencies,[MTA New York City Transit]
1,timezone,America/New_York
2,start_date,20250323
3,end_date,20250518
4,num_routes,30
5,num_trips,20298
6,num_stops,1497
7,num_shapes,311
8,sample_date,20250327
9,num_routes_active_on_sample_date,28


In [26]:
subway_data.gk_feed.stops

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,101,Van Cortlandt Park-242 St,40.889248,-73.898583,1,<NA>
1,101N,Van Cortlandt Park-242 St,40.889248,-73.898583,<NA>,101
2,101S,Van Cortlandt Park-242 St,40.889248,-73.898583,<NA>,101
3,103,238 St,40.884667,-73.900870,1,<NA>
4,103N,238 St,40.884667,-73.900870,<NA>,103
...,...,...,...,...,...,...
1492,S30N,Tompkinsville,40.636949,-74.074835,<NA>,S30
1493,S30S,Tompkinsville,40.636949,-74.074835,<NA>,S30
1494,S31,St George,40.643748,-74.073643,1,<NA>
1495,S31N,St George,40.643748,-74.073643,<NA>,S31


In [27]:
# Get all stops without a parent station
stops = subway_data.gk_feed.stops
stops_no_parent = stops[stops.parent_station.isna()]
stops_no_parent.head(10)

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,101,Van Cortlandt Park-242 St,40.889248,-73.898583,1,<NA>
3,103,238 St,40.884667,-73.900870,1,<NA>
6,104,231 St,40.878856,-73.904834,1,<NA>
9,106,Marble Hill-225 St,40.874561,-73.909831,1,<NA>
12,107,215 St,40.869444,-73.915279,1,<NA>
15,108,207 St,40.864621,-73.918822,1,<NA>
18,109,Dyckman St,40.860531,-73.925536,1,<NA>
21,110,191 St,40.855225,-73.929412,1,<NA>
24,111,181 St,40.849505,-73.933596,1,<NA>
27,112,168 St-Washington Hts,40.840556,-73.940133,1,<NA>


In [ ]:
from pathlib import Path
COMMUNITY_DISTRICTS_GEOJSON = Path("demographic_data/community_districts.geojson")
# Load the community districts GeoJSON file
import geopandas as gpd
community_districts = gpd.read_file(COMMUNITY_DISTRICTS_GEOJSON)
# Check the first few rows of the GeoDataFrame
print(community_districts.head())

   OBJECTID  BoroCD   Shape__Area  Shape__Length  \
0         1     308  4.560379e+07   38232.887315   
1         2     410  1.720774e+08  105822.377352   
2         3     206  4.266432e+07   35875.709939   
3         4     480  3.277756e+07   47338.739592   
4         5     104  4.931005e+07   67546.045457   

                                            geometry  
0  POLYGON ((-73.95829 40.67984, -73.95596 40.679...  
1  MULTIPOLYGON (((-73.85722 40.65029, -73.85902 ...  
2  POLYGON ((-73.87185 40.84377, -73.87192 40.843...  
3  POLYGON ((-73.86272 40.76668, -73.86281 40.766...  
4  POLYGON ((-73.99394 40.77319, -73.9937 40.7730...  


In [29]:
# Create a geopandas DataFrame from the subway stops
subway_stops_gdf = gpd.GeoDataFrame(
    stops_no_parent,
    geometry=gpd.points_from_xy(stops_no_parent.stop_lon, stops_no_parent.stop_lat),
    crs="EPSG:4326"
)

print(subway_stops_gdf.head())

   stop_id                  stop_name   stop_lat   stop_lon  location_type  \
0      101  Van Cortlandt Park-242 St  40.889248 -73.898583              1   
3      103                     238 St  40.884667 -73.900870              1   
6      104                     231 St  40.878856 -73.904834              1   
9      106         Marble Hill-225 St  40.874561 -73.909831              1   
12     107                     215 St  40.869444 -73.915279              1   

   parent_station                    geometry  
0            <NA>  POINT (-73.89858 40.88925)  
3            <NA>  POINT (-73.90087 40.88467)  
6            <NA>  POINT (-73.90483 40.87886)  
9            <NA>  POINT (-73.90983 40.87456)  
12           <NA>  POINT (-73.91528 40.86944)  


In [30]:
import plotly.express as px
from src.underserved_areas import generate_uniform_points_by_count, compute_dist_to_nearest_point

N_SAMPLES = 10000

sampled_points = generate_uniform_points_by_count(community_districts, n_points=N_SAMPLES)

sampled_points["lon"] = sampled_points.geometry.x
sampled_points["lat"] = sampled_points.geometry.y

sampled_points = compute_dist_to_nearest_point(sampled_points, subway_stops_gdf)

# Make a scatter map where the color of the points is based on the distance to the nearest subway stop
# map_nyc = px.scatter_map(
#     sampled_points,
#     lat="lat",
#     lon="lon",
#     color="Distance to Nearest Stop (km)",
#     color_continuous_scale="redor",
#     zoom=9,
#     height=600,
#     width=800,
#     title="Distance to Nearest Subway Stop"
# )

# # Plot the subway stops on the map
# map_nyc.add_trace(
#     px.scatter_map(
#         subway_stops_gdf,
#         lat="stop_lat",
#         lon="stop_lon",
#         hover_name="stop_name",
#         color_discrete_sequence=["blue"],
#         opacity=0.5,
#     ).data[0]
# )

# # Update the layout to use OpenStreetMap style
# map_nyc.update_layout(
#     title_text="Distance to Nearest Subway Stop", 
#     mapbox_style="open-street-map",
# )

# map_nyc.show()

# Conjecture
Community districts with high population density, on average, have lower incomes.

In [31]:
from src.load_demographic_data import load_demographic_data, DEMOGRAPHIC_DATASETS

# Load demographic data
load_demographic_data()

# Get the median income and population datasets
median_income_data = DEMOGRAPHIC_DATASETS[0]
total_population_data = DEMOGRAPHIC_DATASETS[1]

print()

# Check the first few rows of the median income dataset
print(median_income_data.dataframe.head())

# Check the columns of the median income dataset
print(median_income_data.dataframe.columns)

# Check the first few rows of the total population dataset
print(total_population_data.dataframe.head())

# Check the columns of the total population dataset
print(total_population_data.dataframe.columns)

Loaded Median Incomes from demographic_data/median_incomes.csv
Loaded Total Population from demographic_data/total_population.csv
Loaded Total Population by Race Ethnicity from demographic_data/total_population_by_race_ethnicity.csv
Loaded Total Population by Age Group from demographic_data/total_population_by_age_group.csv

               Location             Household Type  TimeFrame DataFormat  \
0               Astoria             All Households       2023    Dollars   
1               Astoria                   Families       2023    Dollars   
2               Astoria     Families with Children       2023    Dollars   
3               Astoria  Families without Children       2023    Dollars   
4  Battery Park/Tribeca             All Households       2023    Dollars   

     Data  Fips  
0   84590   401  
1   94918   401  
2   85568   401  
3  110222   401  
4  198945   101  
Index(['Location', 'Household Type', 'TimeFrame', 'DataFormat', 'Data',
       'Fips'],
      dtype='object'

In [32]:
# Get the most recent timeframe for the median income dataset
most_recent_timeframe = median_income_data.dataframe["TimeFrame"].max()
print(f"Most recent timeframe: {most_recent_timeframe}")

# Make sure that there is a row for the most recent timeframe for all locations
# in the median income dataset
recent_timeframe_rows = median_income_data.dataframe[
    median_income_data.dataframe["TimeFrame"] == most_recent_timeframe
]
assert median_income_data.dataframe["Location"].nunique() == recent_timeframe_rows["Location"].nunique(), \
    "Not all locations have a row for the most recent timeframe in the median income dataset."

# Filter the median income dataset to include the following
# - "Household Type" = "All Households"
filtered_median_income = recent_timeframe_rows[
    recent_timeframe_rows["Household Type"] == "All Households"
]

# Check the first few rows of the filtered median income dataset
print(filtered_median_income.head())

Most recent timeframe: 2023
                Location  Household Type  TimeFrame DataFormat    Data  Fips
0                Astoria  All Households       2023    Dollars   84590   401
4   Battery Park/Tribeca  All Households       2023    Dollars  198945   101
8              Bay Ridge  All Households       2023    Dollars   88566   310
12               Bayside  All Households       2023    Dollars  107607   411
16          Bedford Park  All Households       2023    Dollars   42387   207


In [33]:
# Get the most recent timeframe for the median income dataset
most_recent_timeframe = total_population_data.dataframe["TimeFrame"].max()
print(f"Most recent timeframe: {most_recent_timeframe}")

# Make sure that there is a row for the most recent timeframe for all locations
recent_timeframe_rows = total_population_data.dataframe[
    total_population_data.dataframe["TimeFrame"] == most_recent_timeframe
]
assert total_population_data.dataframe["Location"].nunique() == recent_timeframe_rows["Location"].nunique(), \
    "Not all locations have a row for the most recent timeframe in the median income dataset."

filtered_total_population = recent_timeframe_rows

# Check the first few rows of the filtered total population dataset
print(filtered_total_population.head())

Most recent timeframe: 2022
                       Location  TimeFrame DataFormat      Data Fips
2890       Battery Park/Tribeca       2022     Number   68418.0  101
2891          Greenwich Village       2022     Number   80684.0  102
2892            Lower East Side       2022     Number  141787.0  103
2893            Chelsea/Clinton       2022     Number  112582.0  104
2894  Midtown Business District       2022     Number   52570.0  105


In [34]:
# Align the median income and total population datasets on the Location column
filtered_median_income = filtered_median_income.set_index("Location")
filtered_total_population = filtered_total_population.set_index("Location")
filtered_median_income = filtered_median_income.loc[filtered_total_population.index]
filtered_total_population = filtered_total_population.loc[filtered_median_income.index]

In [35]:
import pandas as pd

# Convert the data column to numeric for both datasets
filtered_median_income["Data"] = pd.to_numeric(filtered_median_income["Data"], errors="coerce")
filtered_total_population["Data"] = pd.to_numeric(filtered_total_population["Data"], errors="coerce")

In [36]:
print(filtered_median_income.head())
print(filtered_total_population.head())

                           Household Type  TimeFrame DataFormat    Data  Fips
Location                                                                     
Battery Park/Tribeca       All Households       2023    Dollars  198945   101
Greenwich Village          All Households       2023    Dollars  198945   102
Lower East Side            All Households       2023    Dollars   54482   103
Chelsea/Clinton            All Households       2023    Dollars  122723   104
Midtown Business District  All Households       2023    Dollars  144175   105
                           TimeFrame DataFormat      Data Fips
Location                                                      
Battery Park/Tribeca            2022     Number   68418.0  101
Greenwich Village               2022     Number   80684.0  102
Lower East Side                 2022     Number  141787.0  103
Chelsea/Clinton                 2022     Number  112582.0  104
Midtown Business District       2022     Number   52570.0  105


In [ ]:
from src.load_census_data import load_census_data, get_area_by_fips_code, GEOJSON_DATASET

# Load census data
load_census_data()

In [38]:
fips = filtered_total_population["Fips"]

areas = get_area_by_fips_code(fips)
print(areas)

    Fips  Area (km^2)
0    101     7.224043
1    102     6.119568
2    103     7.579787
3    104     7.758009
4    105     6.365355
5    106     6.956614
6    107     8.792266
7    108     9.449743
8    109     6.897338
9    110     6.183471
10   111    10.704405
11   112    12.629904
12   201    12.354868
13   202     9.922601
14   203     7.464240
15   204     9.358614
16   205     6.429054
17   206     7.111113
18   207     9.491898
19   208    13.452351
20   209    18.185304
21   210    27.978013
22   211    16.425201
23   212    24.340185
24   301    21.229758
25   302    12.234741
26   303    12.598806
27   304     9.129683
28   305    26.641602
29   306    13.954589
30   307    16.987039
31   308     7.560612
32   309     7.098861
33   310    17.561566
34   311    16.559264
35   312    16.254589
36   313    14.635916
37   314    12.730436
38   315    20.184914
39   316     8.192347
40   317    15.089275
41   318    40.063597
42   401    25.163524
43   402    21.192367
44   403  

/Users/Deena/Desktop/DS 501/ds-501-cs4/.venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [39]:
# Convert the Fips column to integers
filtered_total_population["Fips"] = filtered_total_population["Fips"].astype(int)

# For filtered total population, join with the areas on Fips
filtered_total_population = filtered_total_population.join(areas.set_index("Fips"), on="Fips")

# Compute population density
filtered_total_population["Population Density"] = (
    filtered_total_population["Data"] / filtered_total_population["Area (km^2)"]
)

In [ ]:
#compare median income to ridership 
#compare population density to ridership 
#load weaver code
#plot median income and regression and correlation coefficient 
#plot population denisty and regression and correlation coefficient 


